## SPARC-4 mini-pipeline tools
***
# <font color='darkblue'>Bias and Flat-field calibrations</font> 

This notebook shows an example for combining a series of bias and flat exposures into the master calibration product of SPARC4 pipeline.

* Eder Martioli, LNA 20 Jun 2023

In [ ]:
import sparc4.product_plots as s4plt
import sparc4.pipeline_lib as s4pipelib
import sparc4.db as s4db

## User inputs and database handling

In [ ]:
# set night directory name
NIGHTDIR = '20230503'
# set raw data root directory
DATADIR =  "/Volumes/Samsung_T5/Data/SPARC4/minidata/"
# set reduced data root directory
REDUCEDDIR = "/Volumes/Samsung_T5/Data/SPARC4/minidata/reduced/"

# set SPARC4 channel
CHANNEL = 4  # 1, 2, 3 or 4

# whether or not to force reduction even if product already exists
FORCE = False

# get SPARC4 pipeline parameters
p = s4pipelib.init_s4_p(datadir=DATADIR,
                        reducedir=REDUCEDDIR,
                        nightdir=NIGHTDIR,
                        channels="{}".format(CHANNEL),
                        print_report=False)

# create database of raw data for reduction
db = s4db.create_db_from_observations(p['filelists'][CHANNEL-1], 
                                      p['DB_KEYS'], 
                                      include_img_statistics=p["INCLUDE_IMG_STATISTICS"], 
                                      include_only_fullframe=p["FULL_FRAMES_ONLY"], 
                                      output=p['s4db_files'][CHANNEL-1])

# detect all detector modes
detector_modes = s4db.get_detector_modes_observed(db, 
                                                  science_only=True,
                                                  detector_keys=p["DETECTOR_MODE_KEYWORDS"])
# get first valid key
mode_key = next(iter(detector_modes))

## Define input list and output product file names

In [ ]:
# create a list of zeros for current detector mode
zero_list = s4db.get_file_list(db, 
                               obstype=p['BIAS_OBSTYPE_KEYVALUE'], 
                               detector_mode=detector_modes[mode_key])

# define file name of master bias 
master_zero = "{}/{}_s4c{}{}_MasterZero.fits".format(p['reduce_directories'][CHANNEL-1],
                                                     NIGHTDIR,
                                                     p['CHANNELS'][CHANNEL-1],
                                                     mode_key)

# create a list of flats for current detector mode
flat_list = s4db.get_file_list(db, 
                               obstype=p['FLAT_OBSTYPE_KEYVALUE'], 
                               detector_mode=detector_modes[mode_key])

# define file name of master flat
master_flat = "{}/{}_s4c{}{}_MasterDomeFlat.fits".format(p['reduce_directories'][CHANNEL-1],
                                                         NIGHTDIR,
                                                         p['CHANNELS'][CHANNEL-1],
                                                         mode_key)

# Run master calibration

In [ ]:
# calculate master bias and save product to FITS
p = s4pipelib.run_master_calibration(p, 
                                     inputlist=zero_list, 
                                     output=master_zero, 
                                     obstype='bias', 
                                     data_dir=DATADIR, 
                                     reduce_dir=p['reduce_directories'][CHANNEL-1], 
                                     force=FORCE)

In [ ]:
# calculate master dome flat and save product to FITS
p = s4pipelib.run_master_calibration(p, 
                                     inputlist=flat_list, 
                                     output=master_flat, 
                                     obstype='flat', 
                                     data_dir=DATADIR, 
                                     reduce_dir=p['reduce_directories'][CHANNEL-1], 
                                     normalize=True, 
                                     force=FORCE)

# Plot results

In [ ]:
# plot master bias
s4plt.plot_cal_frame(master_zero, 
                     percentile=99.5, 
                     combine_rows=True, 
                     combine_cols=True)

In [ ]:
# plot master flat
s4plt.plot_cal_frame(master_flat, 
                     percentile=99.5, 
                     xcut=512, 
                     ycut=512)